In [1]:
# import dependecies 

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

#Config should contain database username as username and database password as password
#import config

NameError: name 'config' is not defined

In [ ]:
#Make sure the following are present in your config file:
#config.username
#config.password
#config.database_name

In [ ]:
# read vaccinations csv

csv_file = "Resources/vaccinations.csv"
vaccinations_df = pd.read_csv(csv_file)
vaccinations_df.head()

In [ ]:
# set columns for first table

vaccination_rate_df = vaccinations_df[['location', 'iso_code', 'date', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'daily_vaccinations' ]].copy()
vaccination_rate_df.head()

In [ ]:
# set columns for second table

vaccination_daily_df = vaccinations_df[['location', 'iso_code', 'date', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred', 'daily_vaccinations_per_million' ]].copy()
vaccination_daily_df.head()

In [ ]:
# group by country for third table 

# vaccination_country_df = vaccinations_df.groupby
# vaccination_country_df.head()# Setting Up Country Reference Table
countries_df = pd.DataFrame({"location": vaccinations_df["location"].unique(), "iso_code": vaccinations_df["iso_code"].unique()})
countries_df.head()

## Region Date Metadata

In [ ]:
# read region_date_metadata.csv

path = "Resources/region_date_metadata.csv"
region_date_df = pd.read_csv(path)
region_date_df.head()

In [ ]:
#Set column for 4th table
region_date_metadata = region_date_df[['Country_Region', 'Province_State', 'Date', 'Recoveries']].copy()
region_date_metadata.head()

In [ ]:
#Groupby the data  as per Country_region and Date
region_date_metadata_groupby = region_date_metadata.groupby(['Country_Region', 'Date']).sum()['Recoveries']
region_date_metadata_groupby = pd.DataFrame(region_date_metadata_groupby)
region_date_metadata_groupby

In [ ]:
#Removing index columns
region_date_metadata_df = region_date_metadata_groupby.reset_index(['Country_Region', 'Date'])
region_date_metadata_df

## Region Metadata

In [2]:
# load csv to pandas dataframe

path2 = "Resources/region_metadata.csv"
region_metadata_df = pd.read_csv(path2)
region_metadata_df.head()

,Country_Region,Province_State,lat,lon,continent,population,area,density
0,Afghanistan,NaN,33.0000,65.0000,Asia,38041754,652230,58.33
1,Albania,NaN,41.1533,20.1683,Europe,2880917,28748,100.21
2,Algeria,NaN,28.0339,1.6596,Africa,43053054,2381741,18.08
3,Andorra,NaN,42.5063,1.5218,Europe,77142,468,164.83
4,Angola,NaN,-11.2027,17.8739,Africa,31825295,1246700,25.53


In [9]:
region_metadata = region_metadata_df.copy()

,Country_Region,Province_State,lat,lon,continent,population,area,density
0,Afghanistan,NaN,33.0000,65.0000,Asia,38041754,652230,58.33
1,Albania,NaN,41.1533,20.1683,Europe,2880917,28748,100.21
2,Algeria,NaN,28.0339,1.6596,Africa,43053054,2381741,18.08
3,Andorra,NaN,42.5063,1.5218,Europe,77142,468,164.83
4,Angola,NaN,-11.2027,17.8739,Africa,31825295,1246700,25.53
...,...,...,...,...,...,...,...,...
315,West Bank and Gaza,NaN,31.6000,34.8000,Asia,5190143,6020,862.15
316,Western Sahara,NaN,24.2155,-12.8858,Africa,567402,266000,2.13
317,Yemen,NaN,15.5527,48.5164,Asia,28498683,527968,53.98
318,Zambia,NaN,-15.4167,28.2833,Africa,17861030,752612,23.73


In [12]:
#groupby 

region_metadata_groupby = region_metadata.groupby(['Country_Region']).sum()['population']
region_metadata_ = pd.DataFrame(region_metadata_groupby)

In [13]:
#reset_index to fit in the SQL table
region_metadata_ = region_metadata_.reset_index('Country_Region')
region_metadata_

,Country_Region,population
0,Afghanistan,38041754
1,Albania,2880917
2,Algeria,43053054
3,Andorra,77142
4,Angola,31825295
...,...,...
182,West Bank and Gaza,5190143
183,Western Sahara,567402
184,Yemen,28498683
185,Zambia,17861030


In [ ]:
# connect to SQL database 

protocol = 'postgresql'
username = config.username
password = config.password
host = 'localhost'
port = 5432
database_name = config.database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [ ]:
# check tables

insp.get_table_names()

In [ ]:
# Push countries lookup table to database
countries_df.to_sql(name="countries",con=engine,if_exists="fail", index=False)

In [ ]:
# push first table to database  

vaccination_rate_df.to_sql(name='vaccinations', con=engine, if_exists='append', index=False)

In [ ]:
# push second table to database  

vaccination_daily_df.to_sql(name='', con=engine, if_exists='append', index=False)

In [ ]:
# push thrid table to database  

# vaccination_country_df.to_sql(name='', con=engine, if_exists='append', index=False)

In [ ]:
# check data upload first table

pd.read_sql_query('select * from ', con=engine).head()

In [ ]:
# check data upload second table

pd.read_sql_query('select * from ', con=engine).head()

In [ ]:
# check data upload third table

pd.read_sql_query('select * from ', con=engine).head()